## 12.3 Moduły i metody dostarczające (provider methods)

Do tej pory cały proces wstrzykiwania był relatywnie prosty. Jeżeli jednak nie mamy dostępu do klasy której instancję chcemy wstrzyknąć (klasy dostępne w zewnętrznych bibliotekach - przykładowo `Notification`, `SharedPreferences`, `RecyclerView`, `Retrofit`) nie możemy zastosować wstrzykiwania przez konstruktor, metodę czy pole - innymi słowy nie możemy modyfikować tej klasy. Musimy zastosować nieco inny mechanizm.

Załóżmy że nasz komputer składa się tylko z cpu i gpu. Klasy pochodzą z zewnętrznych bibliotek, więc nie możemy dokonać żadnej modyfikacji. Konstruktor klasy `Computer` przyjmuje dwa argumenty: `GPU` i `CPU`

In [ ]:
public class Computer {
    private GPU gpu;
    private CPU cpu;

    public Computer(GPU gpu, CPU cpu) {
        this.gpu = gpu;
        this.cpu = cpu;
    }

    public String work(){
        return "working";
    }
}

public class CPU {}
public class GPU {}

W jaki sposób możemy pokazać bibliotece `Dagger2` jak utworzyć obiekty `Computer`, `GPU` i `CPU`, oraz kolejność ich tworzenia? W tym celu wykorzystamy **moduły** - są to zwykłe klasy które są dodane do grafu tworzonego przez `Dagger2`.

Dodajmy metody dostarczające - mamy dwie zależności: `GPU` i `CPU`.

In [ ]:
@Module
public class ComputerModule {
    @Provides
    static GPU provideGpu(){
        return new GPU();
    }
    
    @Provides
    static CPU provideCpu(){
        return new CPU();
    }
}

Adnotacja `@Provides` pozwala oznaczyć metodę dostarczającą. Gdy `Dagger2` potrzebuje obiektu typu `CPU`, odwołuje się do odpowiedniej metody.

Pozostaje dostarczyć metodę tworzącą obiekt typu `Computer`

In [ ]:
@Provides
static Computer provideComputer(GPU gpu, CPU cpu){
    return new Computer(gpu, cpu);
}

Konstruktor `Computer` wymaga podania obiektów `CPU` i `GPU`, ponieważ mamy już metody dostarczające te obiekty, możemy przekazać `cpu` i `gpu` w argumentach metody `provideComputer` - `Dagger2` automatycznie użyje pozostałych metod dostarczających do utworzenia odpowiednich obiektów.

Do interfejsu `ComputerComponent`, przy adnotacji `@Component` musimy podać tablicę modułów

In [ ]:
@Component(modules = ComputerModule.class)
public interface ComputerComponent {
    void inject(MainActivity mainActivity);
}

W głównej aktywności nie wprowadzamy żadnych zmian w statunku do poprzedniego przykładu.

In [ ]:
public class MainActivity extends AppCompatActivity {
    @Inject
    public Computer computer;

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_main);

        ComputerComponent component = DaggerComputerComponent.create();
        component.inject(this);
        TextView textView = findViewById(R.id.textview);
        textView.setText(computer.work());
    }
}

j